In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import os

# Directories
video_embeddings_dir = '/content/drive/MyDrive/Final Embeddings/video_embeddings-10-epochs-b8'
text_embeddings_dir = '/content/drive/MyDrive/Final Embeddings/row_embeddings'
fused_embeddings_dir = '/content/drive/MyDrive/Concat-fusion_final'


In [6]:

# Create directory for fused embeddings if not exists
os.makedirs(fused_embeddings_dir, exist_ok=True)

# Helper functions

def get_base_name_from_text_filename(text_filename):
    """ Extract base name from text filename before '_embedding.npy' """
    return text_filename.split('_embedding.npy')[0]        #extracts the base name from the text filename by removing the '_embedding.npy' suffix.

def find_matching_text_filename(video_filename, text_filenames):
    """ Find a matching text filename based on base name in video filename. """
    video_base_name = video_filename.split('_embeddings.npy')[0]
    for text_filename in text_filenames:
        text_base_name = get_base_name_from_text_filename(text_filename)
        if text_base_name in video_base_name:
            return text_filename
    return None

def find_anomaly_type(filename):
    if 'AbandonedObject' in filename:
        return 'AbandonedObject'
    elif 'Vandalism' in filename:
        return 'Vandalism'
    elif 'Violence' in filename:
        return 'Violence'
    elif 'Normal' in filename:
        return 'Normal'
    else:
        return 'unknown'

# Load video features
video_feature_list = []
video_filenames = []
for file_name in sorted(os.listdir(video_embeddings_dir)):
    if file_name.endswith('.npy'):
        file_path = os.path.join(video_embeddings_dir, file_name)
        video_features = np.load(file_path)
        video_feature_list.append(video_features)
        video_filenames.append(file_name)

# Load text features
text_feature_list = []
text_filenames = []
for file_name in sorted(os.listdir(text_embeddings_dir)):
    if file_name.endswith('.npy'):
        file_path = os.path.join(text_embeddings_dir, file_name)
        text_features = np.load(file_path)
        text_feature_list.append(text_features)
        text_filenames.append(file_name)

# This code:

# Initializes two lists: video_feature_list to store the loaded video features and video_filenames to store their corresponding filenames.
# Iterates over all files in the video_embeddings_dir.
# For each file with a .npy extension, it loads the video features using np.load() and appends the features and filename to the respective lists.

# from sklearn.decomposition import PCA

# # Example of reducing dimensions of video features
# pca = PCA(n_components=768)  # Match the dimensions of text embeddings
# video_features_reduced = pca.fit_transform(video_features)

# def pad_or_truncate(features, target_dim):
#     current_dim = features.shape[-1]
#     if current_dim > target_dim:
#         return features[..., :target_dim]  # Truncate
#     elif current_dim < target_dim:
#         padding = np.zeros((features.shape[0], target_dim - current_dim))
#         return np.concatenate([features, padding], axis=-1)  # Pad
#     return features

# # Example usage
# video_features_aligned = pad_or_truncate(video_features, target_dim=768)


# def align_dimensions(video_features, text_features):
#     video_dim = video_features.shape[-1]
#     text_dim = text_features.shape[-1]

#     if video_dim != text_dim:
#         target_dim = min(video_dim, text_dim)
#         video_features = pad_or_truncate(video_features, target_dim)
#         text_features = pad_or_truncate(text_features, target_dim)

#     return video_features, text_features

# Match and fuse embeddings
fused_embeddings = {}
for video_filename, video_features in zip(video_filenames, video_feature_list):
    matching_text_filename = find_matching_text_filename(video_filename, text_filenames)

    if matching_text_filename:
        text_features = np.load(os.path.join(text_embeddings_dir, matching_text_filename))

        # Align dimensions
        #video_features, text_features = align_dimensions(video_features, text_features)

        # Perform fusion by concatenation
        fused_features = np.concatenate([video_features, text_features], axis=-1)

        # Get anomaly type for naming
        anomaly_type = find_anomaly_type(video_filename)

        # Create fused embedding filename
        base_video_name = video_filename.split('_embeddings.npy')[0]
        fused_filename = f'{anomaly_type}_{base_video_name}_fused.npy'

        # Save the fused embeddings
        np.save(os.path.join(fused_embeddings_dir, fused_filename), fused_features)
        fused_embeddings[fused_filename] = fused_features

print(f'Fused embeddings have been processed and saved to {fused_embeddings_dir}.')


Fused embeddings have been processed and saved to /content/drive/MyDrive/Concat-fusion_final.


In [7]:
import os

def print_number_of_fused_files(fused_embeddings_dir):
    """Print the number of files in the fused embeddings directory."""
    num_files = len([file for file in os.listdir(fused_embeddings_dir) if os.path.isfile(os.path.join(fused_embeddings_dir, file))])
    print(f'There are {num_files} fused embedding files in the directory: {fused_embeddings_dir}')

# Example usage
print_number_of_fused_files(fused_embeddings_dir)


There are 132 fused embedding files in the directory: /content/drive/MyDrive/Concat-fusion_final


In [5]:
import numpy as np
import os

def print_fused_features(fused_embeddings_dir):
    """Print the features of all fused embedding files in the directory."""
    for file_name in sorted(os.listdir(fused_embeddings_dir)):
        if file_name.endswith('_fused.npy'):
            file_path = os.path.join(fused_embeddings_dir, file_name)
            features = np.load(file_path)
            print(f'Features from file {file_name}:')
            print(features)
            print('-' * 40)

# Example usage
#fused_embeddings_dir = 'path_to_fused_embeddings'
print_fused_features(fused_embeddings_dir)

Features from file AbandonedObject_dataAbandonedObjectCopyofao_01.mp4_fused.npy:
[0.58769482 0.05542566 0.50260133 ... 0.00580006 0.00574278 0.00562771]
----------------------------------------
Features from file AbandonedObject_dataAbandonedObjectCopyofao_02.mp4_fused.npy:
[ 0.7039032  -0.0428123   0.68397892 ...  0.35608411  0.00588024
  0.00598166]
----------------------------------------
Features from file AbandonedObject_dataAbandonedObjectCopyofao_03.mp4_fused.npy:
[ 0.7081905  -0.08748905  0.60288602 ...  0.59701061  0.00516434
  0.00498563]
----------------------------------------
Features from file AbandonedObject_dataAbandonedObjectCopyofao_04.mp4_fused.npy:
[0.51714587 0.05604677 0.51718998 ... 0.22712543 0.07301813 0.00488854]
----------------------------------------
Features from file AbandonedObject_dataAbandonedObjectCopyofao_05.mp4_fused.npy:
[ 0.55977595 -0.02288482  0.51548219 ...  0.0051291   0.00507642
  0.00497713]
----------------------------------------
Features 